In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import pandas as pd
import seaborn as sns
sns.set_context('talk')

plt.rcParams['font.family'] = "Ubuntu"
vcolor = '#582c83'

---

In [7]:
def checkdf(df, a, b, c, d, e, f):
    """
    Prints the dataframe according to the the polling station's stadt identifiers
    """
    return df[(df['Land']==a) & (df['Regierungsbezirk']==b) & (df['Kreis']==c) & (df['Verbandsgemeinde']==d) & (df['Gemeinde']==e) & (df['Kennziffer Briefwahlzugehörigkeit']==f)]

---

In [8]:
df = pd.read_csv("./Files/ew19_wbz/ew19_wbz_ergebnisse.csv", skiprows=4, delimiter=';', encoding="ISO-8859-1")
df.columns

Index(['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde', 'Gemeinde',
       'Kennziffer Briefwahlzugehörigkeit', 'Wahlbezirk', 'Bezirksart',
       'Wahlberechtigte (A)', 'Wahlberechtigte ohne Sperrvermerk (A1)',
       'Wahlberechtigte mit Sperrvermerk (A2)',
       'Wahlberechtigte nach § 24 Abs. 2 EuWO (A3)', 'Wähler (B)',
       'Wähler mit Wahlschein (B1)', 'Ungültig', 'Gültig', 'CDU', 'SPD',
       'GRÜNE', 'DIE LINKE', 'AfD', 'CSU', 'FDP', 'FREIE WÄHLER', 'PIRATEN',
       'Tierschutzpartei', 'NPD', 'FAMILIE', 'ÖDP', 'Die PARTEI',
       'Volksabstimmung', 'BP', 'DKP', 'MLPD', 'SGP', 'TIERSCHUTZ hier!',
       'Tierschutzallianz', 'Bündnis C', 'BIG', 'BGE', 'DIE DIREKTE!',
       'DiEM25', 'III. Weg', 'Die Grauen', 'DIE RECHTE', 'DIE VIOLETTEN',
       'LIEBE', 'DIE FRAUEN', 'Graue Panther', 'LKR', 'MENSCHLICHE WELT', 'NL',
       'ÖkoLinX', 'Die Humanisten', 'PARTEI FÜR DIE TIERE',
       'Gesundheitsforschung', 'Volt', 'Ungekürzte Wahlbezirksbezeichnung'],
      dtype='

### Convert polling station identifiers to ints!!!
Assume:  
* '1A'  and other strings are '0'
* '1B' is '0'

In [9]:
df['Kennziffer Briefwahlzugehörigkeit'] = pd.to_numeric(df['Kennziffer Briefwahlzugehörigkeit'], errors='coerce').fillna(0.0).astype('int')
df['Kennziffer Briefwahlzugehörigkeit'].fillna(value=0, inplace=True)


In [5]:
checkdf(df,11,2,1,0,0,0)

NameError: name 'checkdf' is not defined

In [10]:
df['Volt'].sum()

249098

---

In [12]:
lb = pd.read_csv("./Files/ew19_wbz/ew19_wbz_leitband.csv", skiprows=4, delimiter=';', encoding="ISO-8859-1")
lb.dtypes

Satzart                               int64
Land                                  int64
Regierungsbezirk                      int64
Kreis                                 int64
Verbandsgemeinde                      int64
Gemeinde                              int64
Kennziffer Briefwahlzugehörigkeit     int64
Name                                 object
Textkennzeichen                       int64
dtype: object

---

In [142]:
ddf.head()

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Gültig
0,1,0,3,0,0,0,000608,389
1,1,0,3,0,0,0,000609,167
2,1,0,3,0,0,0,000610,649
3,1,0,3,0,0,0,000611,571
4,1,0,3,0,0,0,000612,574


In [146]:
#Remove duplicates by brute force
#ddf.groupby(by=['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde', 'Gemeinde', 'Kennziffer Briefwahlzugehörigkeit', 'Wahlbezirk'])[['Gültig']].first().reset_index()['Gültig'].sum()

37396889

In [13]:
nlb = lb[['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde','Gemeinde', 'Kennziffer Briefwahlzugehörigkeit', 'Name']]

In [14]:
len(df), len(df.drop_duplicates()), len(nlb), len(nlb.drop_duplicates())

(86887, 86887, 13168, 13071)

df['Gültig'].sum(), df[['CDU', 'SPD','GRÜNE', 'DIE LINKE', 'AfD', 'CSU', 'FDP', 'FREIE WÄHLER', 'PIRATEN',
       'Tierschutzpartei', 'NPD', 'FAMILIE', 'ÖDP', 'Die PARTEI',
       'Volksabstimmung', 'BP', 'DKP', 'MLPD', 'SGP', 'TIERSCHUTZ hier!',
       'Tierschutzallianz', 'Bündnis C', 'BIG', 'BGE', 'DIE DIREKTE!',
       'DiEM25', 'III. Weg', 'Die Grauen', 'DIE RECHTE', 'DIE VIOLETTEN',
       'LIEBE', 'DIE FRAUEN', 'Graue Panther', 'LKR', 'MENSCHLICHE WELT', 'NL',
       'ÖkoLinX', 'Die Humanisten', 'PARTEI FÜR DIE TIERE',
       'Gesundheitsforschung', 'Volt']].sum().sum()

Several Towns might have the same code.

In [285]:
nlb[nlb.Name.duplicated()]

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Name
206,9,1,87,0,0,0,Rosenheim
216,9,2,74,0,0,0,Landshut
217,9,2,75,0,0,0,Passau
311,9,3,75,0,0,0,Regensburg
318,9,4,71,0,0,0,Bamberg
...,...,...,...,...,...,...,...
13145,16,0,77,5050,12,9,"Gößnitz, Stadt"
13146,16,0,77,5004,15,4,Haselbach
13158,16,0,77,5051,36,4,Nobitz
13161,16,0,77,5005,42,5,Rositz


In [15]:
nlb[nlb.Name=='Bamberg']

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Name
314,9,4,61,0,0,0,Bamberg
318,9,4,71,0,0,0,Bamberg
9586,9,4,61,0,0,0,Bamberg


In [16]:
df[(df['Land']==9) & (df['Regierungsbezirk']==4) & (df['Verbandsgemeinde']==0) & (df['Gemeinde']==0) & (df['Kennziffer Briefwahlzugehörigkeit']==0)]['Kreis'].unique()

array([61, 62, 63, 64])

In [17]:
nlb.groupby(by=['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde','Gemeinde', 'Kennziffer Briefwahlzugehörigkeit'])['Name'].last()

Land  Regierungsbezirk  Kreis  Verbandsgemeinde  Gemeinde  Kennziffer Briefwahlzugehörigkeit
1     0                 0      0                 0         0                                        Schleswig-Holstein
                        1      0                 0         0                                          Flensburg, Stadt
                        2      0                 0         0                                    Kiel, Landeshauptstadt
                        3      0                 0         0                                        Lübeck, Hansestadt
                        4      0                 0         0                                         Neumünster, Stadt
                                                                                                         ...          
16    0                 77     5051              23        4                                    Langenleuba-Niederhain
                                                 36        4              

In [18]:
len(nlb['Name']), nlb['Name'].nunique()

(13168, 12461)

In [22]:
llb = lb.groupby(by=['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde', 'Gemeinde', 'Kennziffer Briefwahlzugehörigkeit'])[['Name']].first().reset_index()

In [24]:
len(lb), len(llb)

(13168, 13047)

In [25]:
nd = pd.merge(df, llb, how='left', on=['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde', 'Gemeinde', 'Kennziffer Briefwahlzugehörigkeit'])

In [26]:
nd['Gültig'].sum()

37396889

In [27]:
nd['Volt'].sum()

249098

In [28]:
nd.head()

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt,Ungekürzte Wahlbezirksbezeichnung,Name
0,1,0,3,0,0,0,000608,0,1185,1076,...,0,0,0,0,0,1,1,5,NaN,"Lübeck, Hansestadt"
1,1,0,3,0,0,0,000609,0,636,609,...,0,0,0,1,0,0,0,2,NaN,"Lübeck, Hansestadt"
2,1,0,3,0,0,0,000610,0,1623,1399,...,0,0,0,0,2,4,1,7,NaN,"Lübeck, Hansestadt"
3,1,0,3,0,0,0,000611,0,1604,1484,...,1,0,0,0,4,2,1,1,NaN,"Lübeck, Hansestadt"
4,1,0,3,0,0,0,000612,0,1583,1421,...,1,0,0,0,0,1,0,4,NaN,"Lübeck, Hansestadt"


In [37]:
nd.to_csv('./Files/ew19_wbz/ew19_wbz_ergebnisse_mit_Name.csv')

In [39]:
len(nd), len(df)

(86887, 86887)

In [201]:
nd.columns

Index(['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde', 'Gemeinde',
       'Kennziffer Briefwahlzugehörigkeit', 'Wahlbezirk', 'Bezirksart',
       'Wahlberechtigte (A)', 'Wahlberechtigte ohne Sperrvermerk (A1)',
       'Wahlberechtigte mit Sperrvermerk (A2)',
       'Wahlberechtigte nach § 24 Abs. 2 EuWO (A3)', 'Wähler (B)',
       'Wähler mit Wahlschein (B1)', 'Ungültig', 'Gültig', 'CDU', 'SPD',
       'GRÜNE', 'DIE LINKE', 'AfD', 'CSU', 'FDP', 'FREIE WÄHLER', 'PIRATEN',
       'Tierschutzpartei', 'NPD', 'FAMILIE', 'ÖDP', 'Die PARTEI',
       'Volksabstimmung', 'BP', 'DKP', 'MLPD', 'SGP', 'TIERSCHUTZ hier!',
       'Tierschutzallianz', 'Bündnis C', 'BIG', 'BGE', 'DIE DIREKTE!',
       'DiEM25', 'III. Weg', 'Die Grauen', 'DIE RECHTE', 'DIE VIOLETTEN',
       'LIEBE', 'DIE FRAUEN', 'Graue Panther', 'LKR', 'MENSCHLICHE WELT', 'NL',
       'ÖkoLinX', 'Die Humanisten', 'PARTEI FÜR DIE TIERE',
       'Gesundheitsforschung', 'Volt', 'Ungekürzte Wahlbezirksbezeichnung',
       'Name']

In [30]:
nr = nd.iloc[:,[0,1,2,3,4,5,6,-1]]

In [31]:
nr.loc[nr.Name=='Unknown']

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Name


In [33]:
gb = nd.groupby(by=['Name']).sum()

In [35]:
gb['Volt'].sum()

249098

In [34]:
gb.head()

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),...,DIE FRAUEN,Graue Panther,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt
Name,,,,,,,,,,,,,,,,,,,,,
Aach,7,2,35,5007,1,7,0,815,632,183,...,0,2,0,0,0,0,0,0,1,1
"Aach, Stadt",16,6,70,10002,2,0,5,1677,1416,261,...,5,2,2,2,1,2,2,1,1,3
"Aachen, Stadt",975,585,6630,390,390,0,170,177548,136457,41091,...,107,132,113,80,93,122,512,126,85,3400
"Aalen, Stadt",608,76,2736,380076,6688,0,50,48694,38450,10244,...,37,47,29,27,8,8,35,49,25,123
Aarbergen,42,28,273,7,7,0,5,4565,3782,783,...,2,9,1,3,1,2,4,6,3,12


In [36]:
gb.to_csv("./Files/ew19_wbz/groupedVotes.csv")

In [333]:
ddf.iloc[:,0:7]

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk
0,1,0,3,0,0,0,000608
1,1,0,3,0,0,0,000609
2,1,0,3,0,0,0,000610
3,1,0,3,0,0,0,000611
4,1,0,3,0,0,0,000612
...,...,...,...,...,...,...,...
86882,16,0,77,5005,44,5,0004
86883,16,0,77,5009,47,9,0001
86884,16,0,77,5004,48,4,0001
86885,16,0,77,5009,49,9,0001


In [344]:
nlb.drop_duplicates(inplace=True)

<ipython-input-344-11ccfbd35bdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nlb.drop_duplicates(inplace=True)


In [346]:
len(nlb)

13071

In [351]:
checkdf(nlb, 16, 0, 77, 5005, 44, 5)

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Name
13163,16,0,77,5005,44,5,Starkenberg


In [350]:
checkdf(df, 16, 0, 77, 5005, 44, 5)

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,Graue Panther,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt,Ungekürzte Wahlbezirksbezeichnung
86879,16,0,77,5005,44,5,0001,0,827,786,...,0,1,0,0,0,0,3,0,3,NaN
86880,16,0,77,5005,44,5,0002,0,366,338,...,0,2,0,0,1,0,0,0,0,NaN
86881,16,0,77,5005,44,5,0003,0,256,244,...,0,0,0,0,0,0,2,0,0,NaN
86882,16,0,77,5005,44,5,0004,0,165,158,...,1,0,0,0,0,0,1,0,0,NaN


In [349]:
df.tail()

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,Graue Panther,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt,Ungekürzte Wahlbezirksbezeichnung
86882,16,0,77,5005,44,5,0004,0,165,158,...,1,0,0,0,0,0,1,0,0,NaN
86883,16,0,77,5009,47,9,0001,0,448,410,...,0,0,0,0,0,0,0,1,1,NaN
86884,16,0,77,5004,48,4,0001,0,1027,889,...,0,1,0,0,1,0,2,0,0,NaN
86885,16,0,77,5009,49,9,0001,0,254,242,...,0,1,1,0,0,0,1,0,0,NaN
86886,16,0,77,5004,52,4,0001,0,1638,1403,...,5,0,1,0,0,1,6,3,2,NaN


In [352]:
nd = df.merge(nlb, how='left', on=['Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde', 'Gemeinde', 'Kennziffer Briefwahlzugehörigkeit'])

In [354]:
nd.head()

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt,Ungekürzte Wahlbezirksbezeichnung,Name
0,1,0,3,0,0,0,000608,0,1185,1076,...,0,0,0,0,0,1,1,5,NaN,"Lübeck, Hansestadt"
1,1,0,3,0,0,0,000609,0,636,609,...,0,0,0,1,0,0,0,2,NaN,"Lübeck, Hansestadt"
2,1,0,3,0,0,0,000610,0,1623,1399,...,0,0,0,0,2,4,1,7,NaN,"Lübeck, Hansestadt"
3,1,0,3,0,0,0,000611,0,1604,1484,...,1,0,0,0,4,2,1,1,NaN,"Lübeck, Hansestadt"
4,1,0,3,0,0,0,000612,0,1583,1421,...,1,0,0,0,0,1,0,4,NaN,"Lübeck, Hansestadt"


In [359]:
nd.Name.isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
91380    False
91381    False
91382    False
91383    False
91384    False
Name: Name, Length: 91385, dtype: bool

In [362]:
nd.loc[nd.Name.isnull()]

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt,Ungekürzte Wahlbezirksbezeichnung,Name
70034,9,7,62,0,0,00,02,0,1159,909,...,0,0,0,0,0,0,1,1,NaN,NaN
70035,9,7,62,0,0,00,03,0,1207,904,...,0,0,1,0,0,1,1,2,NaN,NaN
70036,9,7,62,0,0,00,04,0,1147,988,...,0,0,0,0,2,3,1,0,NaN,NaN
70037,9,7,62,0,0,00,05,0,1572,1311,...,1,1,0,0,0,0,1,3,NaN,NaN
70038,9,7,62,0,0,00,06,0,1803,1431,...,0,0,0,0,3,2,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86413,15,0,81,455,455,00,23,0,293,259,...,0,1,1,0,0,1,0,0,NaN,NaN
86414,15,0,81,455,455,00,24,0,221,200,...,0,0,0,1,0,1,2,0,NaN,NaN
86415,15,0,81,455,455,00,25,0,134,122,...,0,1,0,0,0,0,1,1,NaN,NaN
86416,15,0,81,455,455,00,26,0,346,307,...,0,0,0,0,0,0,0,1,NaN,NaN


In [365]:
checkdf(df, 9, 7, 62, 0, 0, 0)

,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,Graue Panther,LKR,MENSCHLICHE WELT,NL,ÖkoLinX,Die Humanisten,PARTEI FÜR DIE TIERE,Gesundheitsforschung,Volt,Ungekürzte Wahlbezirksbezeichnung
65535,9,7,62,0,0,0,01,0,2165,1582,...,0,0,1,0,2,2,4,0,8,NaN
